In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import nltk


In [2]:
# Read the url file into the pandas

df = pd.read_excel(r"C:\Users\Amrit\Downloads\Input.xlsx")


# Rows which will be drop in case of scarping gets failed
row_to_drop = []


# To get each row URL and URL_id we will loop it

for row in df.itertuples(index=True):
    url = row.URL
    url_id = row.URL_ID
     
        
    # Header to hit site as disguised
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    
    
    # Making request to the url
    try:
     HTML = requests.get(url,headers=headers).text
    except:
     print(f"Can not get response from {url_id}")
     row_to_drop.append(url_id)
   
   
    # Create a beautifulsoup object
    soup = BeautifulSoup(HTML, 'lxml')
    
    
    # Find the title    
    try: 
     title = soup.find(attrs= {"class":"entry-title"}).get_text()
    except: 
     print(f"No tittle found {url_id}")
     row_to_drop.append(url_id)
       
    
    # Find the article       
    try:
      articles = soup.find(attrs= {"class":"td-post-content tagdiv-type"}).get_text()
      articles= articles.replace('\n','')
    except AttributeError:
      print(f"No article scrap {url_id}")
      row_to_drop.append(url_id)

        
    # Create a new file with url as file name having only title and the article
    file = open("C:\\Users\\Amrit\\Desktop\\text\\"+str(url_id)+".txt","w",encoding="utf-8") 
    file.write(title + '\n' + articles)
    file.close()








No article scrap 2345.0
No article scrap 4279.2
No article scrap 8435.4
No article scrap 11668.0
No article scrap 14900.6
No article scrap 17671.4
No article scrap 33372.6
No article scrap 33834.4
No article scrap 37528.8
No article scrap 40761.4
No article scrap 41223.2
No article scrap 43070.4


In [3]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Amrit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Amrit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Directories of file using in Analysis

text_dir = r"C:\\Users\\Amrit\\Desktop\\text"

stopwords_dir = r"C:\Users\Amrit\Desktop\StopWords-20231015T081423Z-001"

sentiment_dir = r"C:\\Users\\Amrit\\Desktop\\MasterDictionary-20231015T081424Z-001"



# Load all stop words from the stopwords directory and store it in the set variable
stop_words = set()

for files in os.listdir(stopwords_dir):
     f = open(os.path.join(stopwords_dir,files),'r',encoding='ISO-8859-1') 
     stop_words.update(set(f.read().splitlines()))
        

# Load all text files  from the  directory and store it in a list        
art = []


for text_file in os.listdir(text_dir):
  with open(os.path.join(text_dir,text_file),'r',encoding="utf8") as f:
    text = f.read()  
# tokenize the given text file
    words = word_tokenize(text)
    
    
# Remove the stop words from the tokens
    filtered_text = [word for word in words if word.lower() not in stop_words]
    
    
# Add each filtered tokens of each file into a list
    art.append(filtered_text)

    
    
# store positive, Negative words from the directory
Positive=set()
Negative=set()

for files in os.listdir(sentiment_dir):
  if files =='positive-words.txt':
    with open(os.path.join(sentiment_dir,files),'r',encoding='ISO-8859-1') as f:
      Positive.update(f.read().splitlines())
  else:
    with open(os.path.join(sentiment_dir,files),'r',encoding='ISO-8859-1') as f:
      Negative.update(f.read().splitlines())
    
    
# Collecting the positive and negative words from each file and calculating the scores from the positive and negative words     
    
Positive_words = []
Positive_score = []
Negative_words =[]
Negative_score = []
Polarity_score = []
Subjectivity_score = []

#iterate through the list of art
for i in range(len(art)):
  Positive_words.append([word for word in art[i] if word.lower() in Positive])
  Positive_score.append(len(Positive_words[i]))
  Negative_words.append([word for word in art[i] if word.lower() in Negative])
  Negative_score.append(len(Negative_words[i]))
  Polarity_score.append((Positive_score[i] - Negative_score[i]) / ((Positive_score[i] + Negative_score[i]) + 0.000001))
  Subjectivity_score.append((Positive_score[i] + Negative_score[i]) / ((len(art[i])) + 0.000001))
    

In [5]:
# Average Sentence Length = the number of words / the number of sentences
avg_sentence_length = []


# Percentage of Complex words = the number of complex words / the number of words 
Percentage_of_Complex_words  =  []


# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
Fog_Index = []
complex_word_count =  []
avg_syllable_word_count =[]


# Using stopwords we will remove useless data
stopwords = set(stopwords.words('english'))
def measure(file):
  with open(os.path.join(text_dir, file),'r',encoding="utf8") as f:
    text = f.read()
    
    
# Remove punctuations 
    text = re.sub(r'[^\w\s.]','',text)
    
    
# Split the given text file into sentences
    sentences = text.split('.')
    
    
# Total number of sentences in a file
    num_sentences = len(sentences)
    
    
# Total words in the file
    words = [word  for word in text.split() if word.lower() not in stopwords ]
    num_words = len(words)

    
# Complex words having syllable count is greater than 2 and the words in the text that contain more than two syllables.
    complex_words = []
    for word in words:
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word > 2:
        complex_words.append(word)

        
        
# Syllable Count Per Word
# We count the number of Syllables in each word of the text by counting the vowels present in each word.
# We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.
    syllable_count = 0
    syllable_words =[]
    for word in words:
      if word.endswith('es'):
        word = word[:-2]
      elif word.endswith('ed'):
        word = word[:-2]
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word >= 1:
        syllable_words.append(word)
        syllable_count += syllable_count_word


        
    avg_sentence_len = num_words / num_sentences
    avg_syllable_word_count = syllable_count / len(syllable_words)
    Percent_Complex_words  =  len(complex_words) / num_words
    Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

    return avg_sentence_len, Percent_Complex_words, Fog_Index, len(complex_words),avg_syllable_word_count


# iterate through each file or art
for file in os.listdir(text_dir):
  x,y,z,a,b = measure(file)
  avg_sentence_length.append(x)
  Percentage_of_Complex_words.append(y)
  Fog_Index.append(z)
  complex_word_count.append(a)
  avg_syllable_word_count.append(b)
    


In [6]:

# Word Count and Average Word Length Sum of the total number of characters in each word/Total number of words
# We count the total cleaned words present in the text by 
# Removing the stop words (using stopwords class of nltk package).
# removing any punctuations like ? ! , . from the word before counting.

def cleaned_words(file):
  with open(os.path.join(text_dir,file), 'r',encoding="utf8") as f:
    text = f.read()
    text = re.sub(r'[^\w\s]', '' , text)
    words = [word  for word in text.split() if word.lower() not in stopwords]
    length = sum(len(word) for word in words)
    average_word_length = length / len(words)
  return len(words),average_word_length

word_count = []
average_word_length = []
for file in os.listdir(text_dir):
  x, y = cleaned_words(file)
  word_count.append(x)
  average_word_length.append(y)


# To calculate Personal Pronouns mentioned in the text, we use regex to find 
# The counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken so that the country name US is not included in the list.
def count_personal_pronouns(file):
  with open(os.path.join(text_dir,file), 'r',encoding ='utf8') as f:
    text = f.read()
    personal_pronouns = ["I", "we", "my", "ours", "us"]
    count = 0
    for pronoun in personal_pronouns:
      count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
  return count


pp_count = []
for file in os.listdir(text_dir):
  x = count_personal_pronouns(file)
  pp_count.append(x)

In [7]:
output_df = pd.read_excel('Output Data Structure.xlsx',index_col = 0)

# URL_ID 44 ,57, 144 does not exists i,e. page does not exist, throughs 404 error
# so we are going to drop these rows from the table
output_df.drop(row_to_drop, axis = 0, inplace=True)

# These are the required parameters 
variables = [Positive_score,
            Negative_score,
            Polarity_score,
            Subjectivity_score,
            avg_sentence_length,
            Percentage_of_Complex_words,
            Fog_Index,
            avg_sentence_length,
            complex_word_count,
            word_count,
            avg_syllable_word_count,
            pp_count,
            average_word_length]

# write the values to the dataframe
for i, var in enumerate(variables):
  output_df[var] = var

#now save the dataframe to the disk
output_df.to_csv('Final_Output_Data.csv')
     

C:\Users\Amrit\AppData\Local\Temp\ipykernel_17756\909094616.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[var] = var
C:\Users\Amrit\AppData\Local\Temp\ipykernel_17756\909094616.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[var] = var
C:\Users\Amrit\AppData\Local\Temp\ipykernel_17756\909094616.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat

C:\Users\Amrit\AppData\Local\Temp\ipykernel_17756\909094616.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[var] = var
C:\Users\Amrit\AppData\Local\Temp\ipykernel_17756\909094616.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[var] = var
C:\Users\Amrit\AppData\Local\Temp\ipykernel_17756\909094616.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat

C:\Users\Amrit\AppData\Local\Temp\ipykernel_17756\909094616.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[var] = var
C:\Users\Amrit\AppData\Local\Temp\ipykernel_17756\909094616.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[var] = var
C:\Users\Amrit\AppData\Local\Temp\ipykernel_17756\909094616.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat